In [1]:
import json
import json
from typing import List, Union
import itertools

import numpy as np
from collections import defaultdict


In [2]:
def estimate_pass_at_k(
    num_samples: Union[int, List[int], np.ndarray],
    num_correct: Union[List[int], np.ndarray],
    k: int
) -> np.ndarray:
    """
    Estimates pass@k of each problem and returns them in an array.
    """

    def estimator(n: int, c: int, k: int) -> float:
        """
        Calculates 1 - comb(n - c, k) / comb(n, k).
        """
        if n - c < k:
            return 1.0
        return 1.0 - np.prod(1.0 - k / np.arange(n - c + 1, n + 1))

    if isinstance(num_samples, int):
        num_samples_it = itertools.repeat(num_samples, len(num_correct))
    else:
        assert len(num_samples) == len(num_correct)
        num_samples_it = iter(num_samples)

    return np.array([estimator(int(n), int(c), k) for n, c in zip(num_samples_it, num_correct)])


In [10]:
def parse_rescue(text, offset=0):
    # Split the text into sections using "id:" as a delimiter

    sections = text.split('-------------------------')
    
    # Initialize a list to store the parsed data
    parsed_data = []
    
    # Iterate through each section (excluding the first empty section)
    for section in sections:
        # Initialize a dictionary to store the parsed information for this section
        section_data = {}

        current_data = section.split('\n')
        if current_data[0] == '':
            current_data = current_data[1:]
        
        if len(current_data) < 3:
            continue

        
        # Extract the ID
        section_data['id'] = (offset + int(current_data[0].split(":")[-1].strip())-1)//10

        # Extract the status 1
        section_data['pattern'] = current_data[1].strip()
        
        # Extract the status 1
        section_data['status'] = current_data[2].strip()
        
        if section_data['status'] == "RESULT-TRUE":
            section_data['status'] = True
        else:
            section_data['status'] = False
        

        # if section_data['status1'] != "success":
        #     continue

        # Extract the status 1
        # section_data['status2'] = current_data[2].strip()

        # # Extract the time
        # section_data['time'] = current_data[3].strip()

        # # Extract the pattern
        # section_data['pattern'] = current_data[4].strip()

        # Append the parsed data for this section to the list
        parsed_data.append(section_data)
    
    return parsed_data

In [13]:
names = ["Text_DaVinci_Refined_Output","GPT3.5_Raw_Output","GPT3.5_Refined_Output"]
for name in names:
    print(name)
    outfilename = f'./ReDoSHunter_Results/{name}_full_result.txt'

    with open(outfilename, 'r') as f:
        result = f.read()
    redoshunter_data = parse_rescue(result)

    # print(rescue_data[0])


    results = defaultdict(list)

    for r in redoshunter_data:
        results[r['id']].append(r)
    

    # Calculate pass@k.
    total, correct = [], []
    for result in results.values():
            passed = [r["status"] for r in result]
            # print(passed)
            total.append(len(passed))
            correct.append(sum(passed))
    total = np.array(total)
    correct = np.array(correct)

    ks = [1,5,10]
    pass_at_k = {f"pass@{k}": estimate_pass_at_k(total, correct, k).mean()
                        for k in ks if (total >= k).all()}
    print(pass_at_k)

Text_DaVinci_Refined_Output
{'pass@1': 0.03556430446194226, 'pass@5': 0.07322522184726908, 'pass@10': 0.09580052493438321}
GPT3.5_Raw_Output
{'pass@1': 0.035039370078740154, 'pass@5': 0.07655813856601257, 'pass@10': 0.0984251968503937}
GPT3.5_Refined_Output
{'pass@1': 0.03188976377952756, 'pass@5': 0.07124630254551514, 'pass@10': 0.09186351706036745}


In [16]:
# find all text files in a folder
import glob
import os

path = './ReDoSHunter_Results/Text_DaVinci_Raw/'
files = [f for f in glob.glob(path + "**/*.txt", recursive=True)]
filenames = []
for file in files:
    if "only" not in file:
        filenames.append(file)
print(len(filenames))


8


In [ ]:
for file in filenames:
    print(file)
    offset = int(file.split("_")[7])
    # print(offset)
    with open(file, 'r') as f:
        result = f.read()
    redoshunter_data = parse_rescue(result, offset=offset)

    # print(rescue_data[0])


In [21]:


results = defaultdict(list)

for r in redoshunter_data:
        results[r['id']].append(r)
    

    # Calculate pass@k.
total, correct = [], []
for result in results.values():
            passed = [r["status"] for r in result]
            # print(passed)
            total.append(len(passed))
            correct.append(sum(passed))
total = np.array(total)
correct = np.array(correct)

ks = [1,5,10]
pass_at_k = {f"pass@{k}": estimate_pass_at_k(total, correct, k).mean()
                        for k in ks if (total >= k).all()}
print(pass_at_k)

./ReDoSHunter_Results/Text_DaVinci_Raw/Text_DaVinci_Raw_Output_4000_redos_s_java_11111_0_2023_09_10_20_40_36.txt
{'pass@1': 0.03900000000000001, 'pass@5': 0.09781746031746032, 'pass@10': 0.13}
./ReDoSHunter_Results/Text_DaVinci_Raw/Text_DaVinci_Raw_Output_1000_redos_s_java_11111_0_2023_09_10_19_26_05.txt
{'pass@1': 0.028000000000000004, 'pass@5': 0.0680952380952381, 'pass@10': 0.09}
./ReDoSHunter_Results/Text_DaVinci_Raw/Text_DaVinci_Raw_Output_2000_redos_s_java_11111_0_2023_09_10_19_31_52.txt
{'pass@1': 0.012999999999999998, 'pass@5': 0.025, 'pass@10': 0.03}
./ReDoSHunter_Results/Text_DaVinci_Raw/Text_DaVinci_Raw_Output_0_redos_s_java_11111_0_2023_09_10_19_26_02.txt
{'pass@1': 0.039, 'pass@5': 0.0982936507936508, 'pass@10': 0.14}
./ReDoSHunter_Results/Text_DaVinci_Raw/Text_DaVinci_Raw_Output_7000_redos_s_java_11111_0_2023_09_10_20_11_49.txt
{'pass@1': 0.016129032258064516, 'pass@5': 0.06989247311827958, 'pass@10': 0.11290322580645161}
./ReDoSHunter_Results/Text_DaVinci_Raw/Text_DaVinc